# SA Optimizer Diagnosis

The initial run of `sa_optimizer.py` showed no improvement for N=1..10. This is expected because the baseline `submission.csv` is already highly optimized (likely from a public kernel that used `bbox3` or similar).

To verify the SA works, we need to:
1. **Perturb the initial state** significantly (randomize positions) and see if SA can recover a good score.
2. **Run longer**: 10 seconds is very short.
3. **Check N where baseline is weak**: The baseline might be perfect for small N. We should check if we can improve larger N or specific Ns known to be hard.

However, the primary goal is to *replace* `bbox3`. If `bbox3` was used to generate the baseline, we won't beat it by just running a short SA on top of it.

**Hypothesis**: The baseline `submission.csv` we copied is already good. We need to find N values where we can improve.

Let's modify `sa_optimizer.py` to:
1. Accept a list of N to process.
2. Allow "restarts" (random initialization) to escape local optima.
3. Log progress better.

But first, let's verify the "re-implementation" capability. I will create a test where I scramble N=5 and see if SA fixes it.


In [ ]:
import pandas as pd
import numpy as np
from sa_optimizer import ChristmasTree, get_bbox_side, simulated_annealing

# Load baseline
df = pd.read_csv("submission.csv")
df['x'] = df['x'].astype(str).str.replace('s', '')
df['y'] = df['y'].astype(str).str.replace('s', '')
df['deg'] = df['deg'].astype(str).str.replace('s', '')

# Extract N=5
group_data = df[df['id'].str.startswith('005_')]
trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
baseline_score = get_bbox_side(trees)**2 / 5
print(f"Baseline N=5 Score: {baseline_score}")

# Scramble
import random
scrambled_trees = []
for t in trees:
    scrambled_trees.append(ChristmasTree(
        random.uniform(-5, 5), 
        random.uniform(-5, 5), 
        random.uniform(0, 360)
    ))

scrambled_score = get_bbox_side(scrambled_trees)**2 / 5
print(f"Scrambled N=5 Score: {scrambled_score}")

# Run SA on scrambled
print("Running SA on scrambled...")
optimized_trees, optimized_side = simulated_annealing(5, scrambled_trees, max_steps=10000, time_limit=30, initial_temp=2.0)
recovered_score = optimized_side**2 / 5
print(f"Recovered N=5 Score: {recovered_score}")

if recovered_score < scrambled_score:
    print("SUCCESS: SA improved the scrambled state.")
else:
    print("FAILURE: SA did not improve.")
    
if recovered_score <= baseline_score * 1.05: # Allow 5% margin
    print("SUCCESS: SA recovered close to baseline.")
else:
    print("FAILURE: SA stuck in local optimum far from baseline.")
